In [130]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
print(tf.VERSION)
print(keras.__version__)

1.14.0
2.2.4-tf


In [32]:
from tensorflow.keras import backend as K

In [5]:
# create model
model = keras.Sequential()
model.add(layers.Dense(10,activation='relu'))
model.add(layers.Dense(20,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

In [6]:
# config layer
layers.Dense(10,activation=tf.sigmoid)# ''sigmoid
# regularization
layers.Dense(10,activation='relu',kernel_regularizer=keras.regularizers.l1(0.01))#l2(0.01)
# 权重初始化
layers.Dense(10,kernel_initializer='orthogonal')
# bias 初始化
layers.Dense(10,bias_initializer=keras.initializers.constant(3))

In [36]:
# train
model = keras.Sequential([
    layers.Dense(64,input_shape=(32,),activation='relu'),
    layers.Dense(64,activation='relu'),
    layers.Dense(10,activation='sigmoid')
])
model.compile('adam',loss='categorical_crossentropy',metric=['accuracy'])
# tf.train.RMSPropAdam, Adam
# tf.keras.metrics   mean absolute error
# tf.keras.losses     mse, binary_crossentropy,
optimizer = tf.train.AdamOptimizer(0.001)

model.compile(optimizer, 
              loss='mse', # mean_square_error
              metrics=['mae'])# mean absolute error

model.compile(tf.train.RMSPropOptimizer(0.01),
                     loss=keras.losses.categorical_crossentropy,
                     metrics=[keras.metrics.categorical_accuracy])

model.compile('adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [50]:
# import data
import numpy as np

def one_hot_labels_random(shape):
    n,n_class = shape
    classes = np.random.randint(0,n_class,n)
    labels = np.zeros((n, n_class))  
    labels[np.arange(n), classes] = 1
    return labels

data = np.random.random((1000, 32))
labels = one_hot_labels_random((1000, 10))

# model.fit(data, labels, epochs=10,batch_size=64)

In [67]:
valid_data = np.random.random((100,32))
valid_label = one_hot_labels_random((100,10))

# model.fit(data,labels,epochs=10,batch_size=64,validation_data=(valid_data,valid_label))

In [74]:
# tf.inputdata
dataset = tf.data.Dataset.from_tensor_slices((data,labels))
dataset = dataset.batch(32).repeat()


val_dataset = tf.data.Dataset.from_tensor_slices((valid_data,valid_label))
val_dataset = val_dataset.batch(32).repeat()

# model.fit(dataset,epochs=10,steps_per_epoch=30)
# model.fit(dataset,epochs=10,steps_per_epoch=30,validation_data=val_dataset,validation_steps=3)

# evalute & predict
eval_data = np.random.random((100,32))
eval_label = one_hot_labels_random((100,10))
model.evaluate(x=eval_data,y=eval_label,batch_size=32)
model.evaluate(dataset,steps=32)

result = model.predict(eval_data)
result.shape

32/32 [==============================] - 0s 7ms/step - loss: 1.8420 - acc: 0.3320


(100, 10)

In [80]:
# build advance model
input_x = keras.Input(shape=(32,))

x = keras.layers.Dense(64, activation='relu')(input_x)
x = keras.layers.Dense(64, activation='relu')(x)
prdict = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=input_x,outputs=prdict)
model.compile('adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(dataset,epochs=10,steps_per_epoch=30,validation_data=val_dataset,validation_steps=3)

W0710 21:39:39.894055 4376765888 training_utils.py:1300] Expected a shuffled dataset but input dataset `x` is not shuffled. Please invoke `shuffle()` on input dataset.


Epoch 1/10
30/30 [==============================] - 1s 26ms/step - loss: 2.3170 - acc: 0.0990 - val_loss: 2.3127 - val_acc: 0.0938
Epoch 2/10
30/30 [==============================] - 0s 1ms/step - loss: 2.2949 - acc: 0.1239 - val_loss: 2.3111 - val_acc: 0.0521
Epoch 3/10
30/30 [==============================] - 0s 1ms/step - loss: 2.2833 - acc: 0.1335 - val_loss: 2.3136 - val_acc: 0.0833
Epoch 4/10
30/30 [==============================] - 0s 2ms/step - loss: 2.2715 - acc: 0.1474 - val_loss: 2.3164 - val_acc: 0.1042
Epoch 5/10
30/30 [==============================] - 0s 1ms/step - loss: 2.2639 - acc: 0.1571 - val_loss: 2.3175 - val_acc: 0.1042
Epoch 6/10
30/30 [==============================] - 0s 1ms/step - loss: 2.2563 - acc: 0.1741 - val_loss: 2.3180 - val_acc: 0.0938
Epoch 7/10
30/30 [==============================] - 0s 1ms/step - loss: 2.2456 - acc: 0.1741 - val_loss: 2.3197 - val_acc: 0.1146
Epoch 8/10
30/30 [==============================] - 0s 1ms/step - loss: 2.2356 - acc: 0.1

In [85]:
class MyKeras(keras.Model):
    
    def __init__(self,num_classes):
        super(MyKeras,self).__init__(name='mymodel')
        self.layer1 = keras.layers.Dense(32,activation='relu')
        self.layer2 = keras.layers.Dense(num_classes,activation='sigmoid')
    def call(self,inputs):
        x = self.layer1(inputs)
        return  self.layer2(x)
    
    def compute_output_shape(self,input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)
    

In [88]:
model = MyKeras(10)
model.compile('adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(data,labels,batch_size=32,epochs=10)

Epoch 1/10
1000/1000 [==============================] - 0s 221us/sample - loss: 2.3362 - acc: 0.1000
Epoch 2/10
1000/1000 [==============================] - 0s 42us/sample - loss: 2.3134 - acc: 0.1020
Epoch 3/10
1000/1000 [==============================] - 0s 42us/sample - loss: 2.3057 - acc: 0.0960
Epoch 4/10
1000/1000 [==============================] - 0s 42us/sample - loss: 2.3010 - acc: 0.1000
Epoch 5/10
1000/1000 [==============================] - 0s 43us/sample - loss: 2.2987 - acc: 0.1110
Epoch 6/10
1000/1000 [==============================] - 0s 37us/sample - loss: 2.2950 - acc: 0.1080
Epoch 7/10
1000/1000 [==============================] - 0s 40us/sample - loss: 2.2932 - acc: 0.1210
Epoch 8/10
1000/1000 [==============================] - 0s 44us/sample - loss: 2.2908 - acc: 0.1350
Epoch 9/10
1000/1000 [==============================] - 0s 38us/sample - loss: 2.2887 - acc: 0.1310
Epoch 10/10
1000/1000 [==============================] - 0s 43us/sample - loss: 2.2870 - acc: 0.130

# Define layers

- build: Create the weights of the layer. Add weights with the add_weight method.
- call: Define the forward pass.
- compute_output_shape: Specify how to compute the output shape of the layer given the input shape.
- Optionally, a layer can be serialized by implementing the get_config method and the from_config class method.

In [111]:
class MyLayer(keras.layers.Layer):
    
    def __init__(self,output_dim,**kwargs):
        self.output_dim = output_dim
        
        super(MyLayer,self).__init__(**kwargs)
            
#             super(MyLayer, self).__init__(**kwargs)


    def call(self,inputs):
        return tf.matmul(inputs,self.kernel)
    
    def build(self,input_shape):
        shape = tf.TensorShape((input_shape[1], self.output_dim))
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel',
                                  shape=shape,
                                  initializer='uniform',
                                  trainable=True)
        # Make sure to call the `build` method at the end
        super(MyLayer, self).build(input_shape)
    
    def compute_output_shape(self,input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape)
    
    def get_config(self):
        base_config = super(MyLayer,self).get_confg()
        base_config['re'] = self.output_dim
        return base_config
    @classmethod
    def from_config(cls, confg):
        return cls(**confg)
    
# class MyLayer(layers.Layer):

#     def __init__(self, output_dim, **kwargs):
#         self.output_dim = output_dim
#         super(MyLayer, self).__init__(**kwargs)

#     def build(self, input_shape):
#         shape = tf.TensorShape((input_shape[1], self.output_dim))
#         # Create a trainable weight variable for this layer.
#         self.kernel = self.add_weight(name='kernel',
#                                       shape=shape,
#                                       initializer='uniform',
#                                       trainable=True)
#         # Make sure to call the `build` method at the end
#         super(MyLayer, self).build(input_shape)

#     def call(self, inputs):
#         return tf.matmul(inputs, self.kernel)

#     def compute_output_shape(self, input_shape):
#         shape = tf.TensorShape(input_shape).as_list()
#         shape[-1] = self.output_dim
#         return tf.TensorShape(shape)

#     def get_config(self):
#         base_config = super(MyLayer, self).get_config()
#         base_config['re'] = self.output_dim
#         return base_config

#     @classmethod
#     def from_config(cls, config):
#         return cls(**config)

In [112]:
# model
model = keras.Sequential([MyLayer(10), keras.layers.Activation('softmax')])
model.compile('adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(data,labels,batch_size=32,epochs=10)


W0711 21:02:42.258763 4376765888 deprecation.py:506] From /Users/xywy/anaconda3/envs/object_detection/lib/python3.6/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/10
1000/1000 [==============================] - 0s 188us/sample - loss: 2.3067 - acc: 0.0850
Epoch 2/10
1000/1000 [==============================] - 0s 30us/sample - loss: 2.2991 - acc: 0.0970
Epoch 3/10
1000/1000 [==============================] - 0s 33us/sample - loss: 2.2976 - acc: 0.1090
Epoch 4/10
1000/1000 [==============================] - 0s 29us/sample - loss: 2.2959 - acc: 0.1060
Epoch 5/10
1000/1000 [==============================] - 0s 32us/sample - loss: 2.2940 - acc: 0.1120
Epoch 6/10
1000/1000 [==============================] - 0s 28us/sample - loss: 2.2911 - acc: 0.1120
Epoch 7/10
1000/1000 [==============================] - 0s 30us/sample - loss: 2.2888 - acc: 0.1210
Epoch 8/10
1000/1000 [==============================] - 0s 30us/sample - loss: 2.2864 - acc: 0.1260
Epoch 9/10
1000/1000 [==============================] - 0s 29us/sample - loss: 2.2845 - acc: 0.1320
Epoch 10/10
1000/1000 [==============================] - 0s 28us/sample - loss: 2.2823 - acc: 0.140

In [114]:
# tf.keras.callbacks.ModelCheckpoint: Save checkpoints of your model at regular intervals.
# tf.keras.callbacks.LearningRateScheduler: Dynamically change the learning rate.
# tf.keras.callbacks.EarlyStopping: Interrupt training when validation performance has stopped improving.
# tf.keras.callbacks.TensorBoard: Monitor the model's behavior using TensorBoard.

callbacks = [
  # Interrupt training if `val_loss` stops improving for over 2 epochs
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
  # Write TensorBoard logs to `./logs` directory
  tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(data,labels,validation_data=(valid_data,valid_label),batch_size=32,epochs=10,callbacks=callbacks)

Train on 1000 samples, validate on 100 samples
Epoch 1/10
1000/1000 [==============================] - 0s 356us/sample - loss: 2.2811 - acc: 0.1360 - val_loss: 2.3202 - val_acc: 0.0800
Epoch 2/10
1000/1000 [==============================] - 0s 35us/sample - loss: 2.2793 - acc: 0.1400 - val_loss: 2.3180 - val_acc: 0.1000
Epoch 3/10
1000/1000 [==============================] - 0s 35us/sample - loss: 2.2774 - acc: 0.1400 - val_loss: 2.3161 - val_acc: 0.1000
Epoch 4/10
1000/1000 [==============================] - 0s 37us/sample - loss: 2.2745 - acc: 0.1400 - val_loss: 2.3202 - val_acc: 0.0700
Epoch 5/10
1000/1000 [==============================] - 0s 34us/sample - loss: 2.2742 - acc: 0.1540 - val_loss: 2.3171 - val_acc: 0.0600


In [136]:
# save and restore 
# weight only=
model = keras.Sequential([layers.Dense(64, activation='relu', input_shape=(32,)), layers.Dense(10,activation='softmax')])
model.compile(tf.train.AdamOptimizer(0.001),loss='categorical_crossentropy',metrics=['accuracy'])
model.save_weights('./weights/my_model')
model.load_weights('./weights/my_model')

model.save_weights('my_model.h5',save_format='h5')
model.load_weights('my_model.h5')
json_string = model.to_json()
import pprint
import json
# pprint.pprint(json.loads(json_string))
# fresh_model = tf.keras.models.model_from_json(json_string)
# yaml_string = fresh_model.to_yaml()
# fresh_model = tf.keras.models.model_from_yaml(yaml_string)


# create model
model = keras.Sequential([layers.Dense(64,activation='relu',input_shape=(32,)), layers.Dense(10,activation='softmax')])
model.compile('rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(data,labels,batch_size=32,epochs=10)
model.save('my_model.h5')
model = keras.models.load_model('my_model.h5')

Epoch 1/10
1000/1000 [==============================] - 0s 283us/sample - loss: 2.3376 - acc: 0.0920
Epoch 2/10
1000/1000 [==============================] - 0s 43us/sample - loss: 2.3100 - acc: 0.1150
Epoch 3/10
1000/1000 [==============================] - 0s 42us/sample - loss: 2.3033 - acc: 0.1180
Epoch 4/10
1000/1000 [==============================] - 0s 39us/sample - loss: 2.2944 - acc: 0.1150
Epoch 5/10
1000/1000 [==============================] - 0s 40us/sample - loss: 2.2867 - acc: 0.1240
Epoch 6/10
1000/1000 [==============================] - 0s 42us/sample - loss: 2.2788 - acc: 0.1250
Epoch 7/10
1000/1000 [==============================] - 0s 40us/sample - loss: 2.2713 - acc: 0.1450
Epoch 8/10
1000/1000 [==============================] - 0s 41us/sample - loss: 2.2666 - acc: 0.1400
Epoch 9/10
1000/1000 [==============================] - 0s 39us/sample - loss: 2.2595 - acc: 0.1400
Epoch 10/10
1000/1000 [==============================] - 0s 40us/sample - loss: 2.2557 - acc: 0.143

In [139]:
model = keras.Sequential([layers.Dense(64,input_shape=(32,), activation='relu'), layers.Dense(10,activation='softmax')])
model.compile('rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
estimator = tf.keras.estimator.model_to_estimator(model)

W0711 21:40:21.051130 4376765888 estimator.py:1811] Using temporary folder as model directory: /var/folders/wp/_3j2hkz96tg5_qbz2f_47yzc0000gn/T/tmpmzy4ye3o


In [147]:
model = tf.keras.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10,)))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = tf.train.GradientDescentOptimizer(0.2)

model.compile(loss='binary_crossentropy', optimizer=optimizer)
model.summary()


def input_fn():
    data = np.random.random((1024,10))
    labels = np.random.randint(2,size=(1024,1))
    x = tf.cast(data, tf.float32)
    dataset = tf.data.Dataset.from_tensor_slices((x,labels))
    dataset = dataset.repeat(10).batch(32)
    return dataset

strategy = tf.contrib.distribute.MirroredStrategy()
config = tf.estimator.RunConfig(train_distribute=strategy)

keras_estimator = tf.keras.estimator.model_to_estimator(
  keras_model=model,
  config=config,
  model_dir='/tmp/model_dir')

keras_estimator.train(input_fn=input_fn, steps=10)

W0711 21:53:01.454910 4376765888 cross_device_ops.py:1182] Not all devices in `tf.distribute.Strategy` are visible to TensorFlow.


Model: "sequential_55"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_132 (Dense)            (None, 16)                176       
_________________________________________________________________
dense_133 (Dense)            (None, 1)                 17        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


ValueError: Tensor dense_132/bias is not found in /tmp/model_dir/keras/keras_model.ckpt checkpoint {'global_step': [], 'dense_121/bias': [1], 'dense_121/kernel': [16, 1], 'dense_120/kernel': [10, 16], 'dense_120/bias': [16]}